In [ ]:
import os
import re
import json
import time
import math
import requests
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
from rapidfuzz import process, fuzz
from util.logging import setup_logger

# ===== 로깅 =====
logger = setup_logger(__name__)

# ----------------------------------------------------
# 🔹 1. 남동발전 발전소 지역 매핑 크롤러
# ----------------------------------------------------
def crawl_mapping():
    URL = "https://www.koenergy.kr/kosep/hw/fr/ov/ovhw25/main.do?menuCd=FN060202"
    logging.info(f"크롤링 시작: {URL}")

    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(URL, headers=headers, timeout=10)
    res.raise_for_status()
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")

    tables = soup.find_all("table", class_="table_list2")
    if len(tables) < 2:
        logging.error("❌ class='table_list2' 테이블이 2개 이상 존재하지 않습니다.")
        raise RuntimeError("class='table_list2' 테이블이 충분하지 않습니다.")
    table = tables[1]

    mapping = {}
    for tr in table.find_all("tr"):
        tds = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(tds) < 2:
            continue
        사업명, 지역 = tds[0], tds[1]
        if "태양광" not in 사업명:
            continue
        발전구분 = 사업명.replace("발전소", "").replace(" ", "").strip()
        mapping[발전구분] = 지역

    os.makedirs(os.path.dirname(CACHE_JSON), exist_ok=True)
    with open(CACHE_JSON, "w", encoding="utf-8") as f:
        json.dump(mapping, f, ensure_ascii=False, indent=2)

    logging.info(f"{len(mapping)}개 항목 크롤링 완료 → {CACHE_JSON}")
    return mapping


# ----------------------------------------------------
# 🔹 2. CSV 유틸
# ----------------------------------------------------
def sniff_delimiter(path):
    with open(path, "rb") as f:
        raw = f.read(2048)
    text = raw.decode("utf-8", errors="ignore")
    return "," if text.count(",") >= text.count("\t") else "\t"


def read_csv_safe(path):
    delim = sniff_delimiter(path)
    try:
        return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
    except UnicodeDecodeError:
        logging.warning(f"{path} UTF-8 실패 → cp949로 재시도")
        return pd.read_csv(path, encoding="cp949", delimiter=delim, index_col=False)


def collect_files():
    files = []
    for y in YEARS:
        folder = os.path.join(BASE_DIR, y)
        files += glob(os.path.join(folder, "*.csv")) + glob(os.path.join(folder, "*.CSV"))
    return files


def normalize_columns(df):
    df.columns = df.columns.str.strip()
    if "발전구분" not in df.columns:
        expected = ["발전구분", "호기", "일자"] + [f"{i}시 발전량(MWh)" for i in range(1, 25)] + \
                   ["총량(KW)", "평균(KW)", "최대(시간별)", "최소(시간별)", "최대", "최소"]
        df = df.iloc[:, :len(expected)]
        df.columns = expected
    df["발전구분"] = df["발전구분"].astype(str).str.strip()
    df["일자"] = pd.to_datetime(df["일자"], errors="coerce")
    return df


def get_hour_cols(df):
    pat = re.compile(r"^\s*(\d{1,2})시")
    return [c for c in df.columns if pat.search(c)]


# ----------------------------------------------------
# 🔹 3. CSV 통합 및 호기 합산
# ----------------------------------------------------
files = collect_files()
if not files:
    logging.error("⚠️ CSV 파일이 없습니다. 폴더 구조를 확인하세요.")
    raise FileNotFoundError("⚠️ CSV 파일이 없습니다.")

frames = []
for f in files:
    try:
        tmp = read_csv_safe(f)
        tmp = normalize_columns(tmp)
        tmp["파일출처"] = os.path.basename(f)
        frames.append(tmp)
        logging.info(f"불러옴: {os.path.basename(f)} ({len(tmp)}행)")
    except Exception as e:
        logging.warning(f"⚠️ {os.path.basename(f)} 읽기 실패: {e}")

df = pd.concat(frames, ignore_index=True)
before = len(df)
df = df.drop_duplicates()
logging.info(f"🧹 완전 동일행 중복 제거: {before} → {len(df)}")

hour_cols = get_hour_cols(df)
long_df = df.melt(
    id_vars=["발전구분", "호기", "일자"],
    value_vars=hour_cols,
    var_name="시간대",
    value_name="발전량(MWh)"
)
long_df["시간"] = long_df["시간대"].str.extract(r"(\d{1,2})").astype(int)
long_df["일시"] = pd.to_datetime(long_df["일자"], errors="coerce") + pd.to_timedelta(long_df["시간"] - 1, "h")

hoqi_sum = (
    long_df.groupby(["발전구분", "일자", "시간"], as_index=False)["발전량(MWh)"]
           .sum(numeric_only=True)
           .rename(columns={"발전량(MWh)": "합산발전량(MWh)"})
)
hoqi_sum["일시"] = pd.to_datetime(hoqi_sum["일자"], errors="coerce") + pd.to_timedelta(hoqi_sum["시간"] - 1, "h")
hoqi_sum = hoqi_sum[["일시", "발전구분", "합산발전량(MWh)"]]
hoqi_sum = hoqi_sum.sort_values(["발전구분", "일시"]).reset_index(drop=True)


# ----------------------------------------------------
# 🔹 4. 발전소명 → 지역 매핑 + 보정테이블 적용
# ----------------------------------------------------
if os.path.exists(CACHE_JSON):
    logging.info(f"캐시 파일 발견 → {CACHE_JSON}")
    with open(CACHE_JSON, "r", encoding="utf-8") as f:
        mapping = json.load(f)
else:
    logging.info("캐시 파일 없음 → 새로 크롤링 시작")
    mapping = crawl_mapping()

default_region_fix = {...}  # 기존 default_region_fix 그대로 유지

if not os.path.exists(REGION_FIX_JSON):
    os.makedirs(os.path.dirname(REGION_FIX_JSON), exist_ok=True)
    with open(REGION_FIX_JSON, "w", encoding="utf-8") as f:
        json.dump(default_region_fix, f, ensure_ascii=False, indent=2)
    logging.info(f"지역 보정 테이블 생성 완료 → {REGION_FIX_JSON}")
else:
    logging.info(f"기존 지역 보정 테이블 사용 → {REGION_FIX_JSON}")

with open(REGION_FIX_JSON, "r", encoding="utf-8") as f:
    region_fix = json.load(f)

# ... 이하 fast_match, find_nearest_station, get_latlng 등도 동일하게 print→logging.info 변환 ...

logging.info(f"최종 완료: {OUT_MERGED}")

C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)


불러옴: 남동발전량_2022_01.csv (682행)
불러옴: 남동발전량_2022_02.csv (616행)
불러옴: 남동발전량_2022_03.csv (682행)
불러옴: 남동발전량_2022_04.csv (660행)
불러옴: 남동발전량_2022_05.csv (682행)


C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  r

불러옴: 남동발전량_2022_06.csv (660행)
불러옴: 남동발전량_2022_07.csv (682행)
불러옴: 남동발전량_2022_08.csv (682행)
불러옴: 남동발전량_2022_09.csv (660행)
불러옴: 남동발전량_2022_10.csv (682행)


C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  r

...

불러옴: 남동발전량_2024_03.csv (713행)
불러옴: 남동발전량_2024_04.csv (690행)
불러옴: 남동발전량_2024_05.csv (705행)


C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  r

불러옴: 남동발전량_2024_06.csv (660행)
불러옴: 남동발전량_2024_07.csv (682행)
불러옴: 남동발전량_2024_08.csv (682행)
불러옴: 남동발전량_2024_09.csv (659행)
불러옴: 남동발전량_2024_10.csv (694행)


C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return pd.read_csv(path, encoding="utf-8", delimiter=delim, index_col=False)
C:\Users\rlarn\AppData\Local\Temp\ipykernel_6484\1611909873.py:72: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  r

불러옴: 남동발전량_2024_11.csv (690행)
불러옴: 남동발전량_2024_12.csv (713행)
불러옴: 남동발전량_2024_01.csv (713행)
불러옴: 남동발전량_2024_02.csv (667행)
불러옴: 남동발전량_2024_03.csv (713행)
불러옴: 남동발전량_2024_04.csv (690행)
불러옴: 남동발전량_2024_05.csv (705행)
불러옴: 남동발전량_2024_06.csv (660행)
불러옴: 남동발전량_2024_07.csv (682행)
불러옴: 남동발전량_2024_08.csv (682행)
불러옴: 남동발전량_2024_09.csv (659행)
불러옴: 남동발전량_2024_10.csv (694행)
불러옴: 남동발전량_2024_11.csv (690행)
불러옴: 남동발전량_2024_12.csv (713행)
🧹 완전 동일행 중복 제거: 48900 → 24450
🔍 캐시 파일 없음 → 새로 크롤링 시작
🌐 크롤링 시작: https://www.koenergy.kr/kosep/hw/fr/ov/ovhw25/main.do?menuCd=FN060202
✅ 35개 항목 크롤링 완료 → C:/ESG_Project1/merge/cache/namdong_mapping.json
🆕 지역 보정 테이블 생성 완료 → C:/ESG_Project1/merge/cache/region_fix.json

🌍 카카오 API로 지역 좌표 조회 및 최근접 기상관측소 매핑 시작...
 - 강원특별자치도 강릉시: (37.7521116823526, 128.875906235799) → 최근접지점 강릉 (1.33 km)
 - 경상남도 고성군: (34.9730618412549, 128.32236130677) → 최근접지점 통영 (17.55 km)
 - 경상남도 진주시: (35.1802228341536, 128.107706335688) → 최근접지점 진주 (3.3 km)
 - 경상남도 창원시 마산합포구 해운동: (35.1799611525258, 128.563558712827)

In [ ]:
import os
import pandas as pd
from glob import glob

# ----------------------------------------------------
# 🔹 로거 설정
# ----------------------------------------------------
logger = setup_logger(__name__)

# ----------------------------------------------------
# 🔹 경로 설정
# ----------------------------------------------------
BASE_DIR = "C:/ESG_Project1/file/"
SOLAR_CSV = os.path.join(BASE_DIR, "solar_data_file/남동발전량_지역매핑_시간행.csv")
WEATHER_DIR = os.path.join(BASE_DIR, "KMA/")
OUT_CSV = os.path.join(BASE_DIR, "solar_data_file/남동발전량_지역매핑_시간행_기상병합.csv")

# ----------------------------------------------------
# 🔹 CSV 읽기 유틸
# ----------------------------------------------------
def sniff_delimiter(path):
    with open(path, "rb") as f:
        raw = f.read(2048)
    text = raw.decode("utf-8", errors="ignore")
    return "," if text.count(",") >= text.count("\t") else "\t"

def read_csv_safe(path):
    delim = sniff_delimiter(path)
    try:
        return pd.read_csv(path, encoding="utf-8", delimiter=delim)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="cp949", delimiter=delim)

# ----------------------------------------------------
# 🔹 1. 발전량 데이터 불러오기
# ----------------------------------------------------
logger.info(f"🔹 발전량 데이터 불러오기: {SOLAR_CSV}")
solar = read_csv_safe(SOLAR_CSV)
solar["일시"] = pd.to_datetime(solar["일시"], errors="coerce")

# ----------------------------------------------------
# 🔹 2. 기상 데이터 통합
# ----------------------------------------------------
weather_files = sorted(glob(os.path.join(WEATHER_DIR, "OBS_ASOS_TIM_20*.csv")))
if not weather_files:
    logger.error("⚠️ 기상 데이터 파일이 없습니다. 경로를 확인하세요.")
    raise FileNotFoundError("⚠️ 기상 데이터 파일이 없습니다. 경로를 확인하세요.")

frames = []
for wf in weather_files:
    try:
        tmp = read_csv_safe(wf)
        tmp["일시"] = pd.to_datetime(tmp["일시"], errors="coerce")
        tmp = tmp[["지점", "일시", "기온(°C)", "강수량(mm)", "일조(hr)", "일사(MJ/m2)"]]
        frames.append(tmp)
        logger.info(f" - 불러옴: {os.path.basename(wf)} ({len(tmp)}행)")
    except Exception as e:
        logger.error(f"⚠️ {os.path.basename(wf)} 읽기 실패: {e}")

weather = pd.concat(frames, ignore_index=True)
logger.info(f"기상데이터 통합 완료: {len(weather):,}행")

# ----------------------------------------------------
# 🔹 3. 병합 (지점번호↔지점, 일시 기준)
# ----------------------------------------------------
merged = pd.merge(
    solar,
    weather,
    left_on=["지점번호", "일시"],
    right_on=["지점", "일시"],
    how="left"
)

merged = merged.drop(columns=["지점"], errors="ignore")

# ----------------------------------------------------
# 🔹 4. NaN → 0 처리
# ----------------------------------------------------
weather_cols = ["기온(°C)", "강수량(mm)", "일조(hr)", "일사(MJ/m2)"]
merged[weather_cols] = merged[weather_cols].fillna(0)

# ----------------------------------------------------
# 🔹 5. 최종 CSV 저장
# ----------------------------------------------------
merged.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
logger.info(f"최종 병합 및 NaN→0 처리 완료 → {OUT_CSV}")

🔹 발전량 데이터 불러오기: C:/ESG_Project1/file/solar_data_file/남동발전량_지역매핑_시간행.csv
 - 불러옴: OBS_ASOS_TIM_2022.csv (831719행)
 - 불러옴: OBS_ASOS_TIM_2023.csv (845510행)
 - 불러옴: OBS_ASOS_TIM_2024.csv (851256행)
 - 불러옴: OBS_ASOS_TIM_2025.csv (634874행)
✅ 기상데이터 통합 완료: 3,163,359행

✅ 최종 병합 및 NaN→0 처리 완료 → C:/ESG_Project1/file/solar_data_file/남동발전량_지역매핑_시간행_기상병합.csv
                   일시    발전구분        지역  지점번호  합산발전량(MWh)  기온(°C)  강수량(mm)  \
0 2022-01-01 00:00:00  경상대태양광  경상남도 진주시   192        0.00    -3.8      0.0   
1 2022-01-01 01:00:00  경상대태양광  경상남도 진주시   192        0.00    -4.2      0.0   
2 2022-01-01 02:00:00  경상대태양광  경상남도 진주시   192        0.00    -4.8      0.0   
3 2022-01-01 03:00:00  경상대태양광  경상남도 진주시   192        0.00    -5.4      0.0   
4 2022-01-01 04:00:00  경상대태양광  경상남도 진주시   192        0.00    -6.0      0.0   
5 2022-01-01 05:00:00  경상대태양광  경상남도 진주시   192        0.00    -6.3      0.0   
6 2022-01-01 06:00:00  경상대태양광  경상남도 진주시   192        0.00    -7.6      0.0   
7 2022-01-01 07:00:00  경상대태양광  경